In [1]:
!pip install chromadb

In [5]:
import pandas as pd

In [16]:
url = "https://raw.githubusercontent.com/jogfx/MLops-exam/main/job_listings_cleaned.csv"
df = pd.read_csv(url)

In [17]:
df.head()

,Entry_id,Title,Description,Published Date,Location,URL
0,h1458941,Administrativ rådgiver og barselsvikar til syg...,Er du vores nye kollega på sygedagpengeområdet...,2024-04-09,Greve,https://www.jobindex.dk/vis-job/h1458941
1,h1458931,Salgsrådgiver – Et job i bæredygtighed og med ...,"Hvis du kan svare ja, så er du måske den vi sø...",2024-04-09,Vanløse eller Aarhus,https://www.jobindex.dk/vis-job/h1458931
2,h1458930,IT System Specialist,Are you motivated by working with leading tech...,2024-04-09,Birkerød,https://www.jobindex.dk/vis-job/h1458930
3,h1458929,Vil du sælge biler? Bliv salgstrainee hos Bjar...,"Vi søger en salgstrainee, der drømmer om at ar...",2024-04-09,Holstebro,https://www.jobindex.dk/vis-job/h1458929
4,h1458928,Brænder du for Dynamics 365 Finance og Power BI?,Bredana Axcite ønsker at styrke teamet af fina...,2024-04-09,Kolding eller Aarhus V,https://www.jobindex.dk/vis-job/h1458928


In [ ]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from spellchecker import SpellChecker
import nltk

In [ ]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\olive\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\olive\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\olive\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
def tokenize_and_lowercase(text):
    return [token.lower() for token in word_tokenize(text)]

def remove_stop_words(tokens):
    return [word for word in tokens if word not in stop_words]

def lemmatize_words(tokens):
    return [lemmatizer.lemmatize(token) for token in tokens]

def preprocess_text(text):
    tokens = tokenize_and_lowercase(text)
    tokens = remove_stop_words(tokens)
    tokens = lemmatize_words(tokens)
    return ' '.join(tokens)

In [ ]:
text_columns = ['Title', 'Description']

for column in text_columns:
    df[column] = df[column].astype(str).apply(preprocess_text)

In [ ]:
df.head()

,Entry_id,Title,Description,Published Date,Location,URL
0,h1458941,administrativ rådgiver og barselsvikar til syg...,er du vores nye kollega på sygedagpengeområdet...,2024-04-09,Greve,https://www.jobindex.dk/vis-job/h1458941
1,h1458931,salgsrådgiver – et job bæredygtighed og med me...,"hvis du kan svare ja , så er du måske den vi s...",2024-04-09,Vanløse eller Aarhus,https://www.jobindex.dk/vis-job/h1458931
2,h1458930,system specialist,motivated working leading technology ? turn pr...,2024-04-09,Birkerød,https://www.jobindex.dk/vis-job/h1458930
3,h1458929,vil du sælge biler ? bliv salgstrainee ho bjar...,"vi søger en salgstrainee , der drømmer om arbe...",2024-04-09,Holstebro,https://www.jobindex.dk/vis-job/h1458929
4,h1458928,brænder du dynamic 365 finance og power bi ?,bredana axcite ønsker styrke teamet af finansk...,2024-04-09,Kolding eller Aarhus V,https://www.jobindex.dk/vis-job/h1458928


In [ ]:
import pandas as pd
import re

# Function to remove special characters from a string
def remove_special_characters(text):
    # Define a regular expression pattern to match non-alphanumeric characters
    pattern = r'[^a-zA-Z0-9\s]'  # Matches any character that is not a letter, digit, or whitespace

    # Use the sub() function from the re module to replace matched characters with an empty string
    clean_text = re.sub(pattern, '', str(text))  # Convert NaN to string before applying the function
    return clean_text

# Read the CSV file into a DataFrame
df = pd.read_csv('job_listings_full.csv')

# Remove special characters from the "Title" column
df['Title'] = df['Title'].apply(remove_special_characters)

# Remove special characters from the "Description" column, handling NaN values
df['Description'] = df['Description'].fillna('').apply(remove_special_characters)

# Save the modified DataFrame back to a CSV file
df.to_csv('job_listings_clean.csv', index=False)

In [ ]:
import csv

In [ ]:
with open('job_listings_cleaned.csv') as f:
    lines = csv.reader(f)

    documents = []
    metadatas = []
    ids = []
    id = 1

    for i, line in enumerate(lines):
        if i == 0:
            continue
    
    documents.append(line[1])
    metadatas.append({'Entry_id': line[0]})
    ids.append(str(id))
    id += 1

FileNotFoundError: [Errno 2] No such file or directory: 'job_listings_cleaned.csv'

In [2]:
import chromadb
chroma_client = chromadb.Client()

In [6]:
collection = chroma_client.create_collection('test')

NameError: name 'chroma_client' is not defined

In [ ]:
collection.add(
    documents=[df.Description],
    metadatas=[df.Title],
    ids=[df.ids]
)

NameError: name 'collection' is not defined